In [167]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import BatchNormalization

es = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

EPOCHS = 1000

In [168]:
df = pd.read_csv('asian_merged.csv', sep=",")

In [169]:
df

,Unnamed: 0,title,review,rating
0,0,Don't dare blink as you may miss something.,"Wong Kar Mun went blind at the age of two, 18 ...",9/10
1,1,Excellent horror film with a weak ending,Mun (Angelica Lee) is a young woman who has be...,7/10
2,2,A Corneal Transplant from the Twilight Zone.,This is not the first time that a movie where ...,8/10
3,3,"One of the year's best horror movies, though t...",Even the website of this movie gave me the cre...,8/10
4,4,Good and scary!!!,"Of all the horror movie genres in existence, g...",10/10
...,...,...,...,...
1480,269,A rewarding experience,A Tale Of Two Sisters is a heart-wrenching sto...,10/10
1481,270,"Creepy, Cringe, Disturbing,Twisted",Endless expectation for something to happen. C...,6/10
1482,271,It will make you scream like a teenage girl.,I saw this movie while I was in London a few m...,9/10
1483,272,Not as good as I hoped,I watched this film as part of the EXTREME ASI...,10/10


In [170]:
df = df.drop('Unnamed: 0', axis=1)

In [171]:
df

,title,review,rating
0,Don't dare blink as you may miss something.,"Wong Kar Mun went blind at the age of two, 18 ...",9/10
1,Excellent horror film with a weak ending,Mun (Angelica Lee) is a young woman who has be...,7/10
2,A Corneal Transplant from the Twilight Zone.,This is not the first time that a movie where ...,8/10
3,"One of the year's best horror movies, though t...",Even the website of this movie gave me the cre...,8/10
4,Good and scary!!!,"Of all the horror movie genres in existence, g...",10/10
...,...,...,...
1480,A rewarding experience,A Tale Of Two Sisters is a heart-wrenching sto...,10/10
1481,"Creepy, Cringe, Disturbing,Twisted",Endless expectation for something to happen. C...,6/10
1482,It will make you scream like a teenage girl.,I saw this movie while I was in London a few m...,9/10
1483,Not as good as I hoped,I watched this film as part of the EXTREME ASI...,10/10


In [172]:
df['review'][88]

'Let\'s put aside the fact that this film has some creepy moments but zero actual scares. Sorry, I jumped all throughout The Grudge and I knew while I was jumping that it was a terrible film. This didn\'t scare me. Noroi? Crapped my pants. This? I was laughing two thirds of the way through. Why?Because I just might nominate this film for most contrived ending of all time. Even disregarding the sheer amount of plot devices this film borrowed (Angel Heart and Jacob\'s Ladder are all over this film), the last third was absolutely dumbfounding. My jaw was on the floor from the sheer melodrama of it all. I saw this film on lists with Ju-on, Ringu, and a few other notable J-scarers, but boy, if this scared you, Ringu will literally kill you. For the record, Ringu didn\'t scare me either. Not like the American remake did. But that\'s a whole other can of worms.The Eye suffers from a serious dollop of schmaltz. Romantic subplot? Check. Ridiculously cheery classical piano music? Check. A "thing

In [173]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from tqdm import tqdm
from nltk import sent_tokenize,word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [174]:
main = []

# Storing all punctuations using RE library like !;,"% etc
re_puncs = re.compile('[%s]' % re.escape(string.punctuation))
# Storing all stop words like a, an, the, when, there, this etc
stop_word  = set(stopwords.words('english'))
stop_word.add("im")
# print(stop_word)


In [175]:
sentences_listed = [line.split(" ") for line in df['review']]

import gensim

word_model = gensim.models.Word2Vec(sentences = sentences_listed, vector_size = 100)
words = list(word_model.wv.key_to_index)
print(len(words))

5342


In [176]:
!pip install gensim

In [177]:
word_model.wv.most_similar("actress")

[('Hitomi', 0.9967216849327087),
 ("She's", 0.9964917302131653),
 ('act', 0.9958080649375916),
 ('–', 0.9953577518463135),
 ('reveal', 0.9950501322746277),
 ('shown', 0.9949042201042175),
 ('stars;', 0.9948264360427856),
 ("child's", 0.9947243332862854),
 ('freaky', 0.9946986436843872),
 ('sacrifice', 0.9946568608283997)]

In [178]:
word_model.wv.most_similar("death")

[('divorce', 0.9951028823852539),
 ('custody', 0.9939340353012085),
 ('herself', 0.9938431978225708),
 ('tape,', 0.9933721423149109),
 ('Jane', 0.9929178953170776),
 ('journalist', 0.9927891492843628),
 ('search', 0.9925683736801147),
 ('Ryuji', 0.9925025105476379),
 ('sight', 0.9921193718910217),
 ('visit', 0.9916483163833618)]

In [179]:
word_model.wv.most_similar("scary")

[('quite', 0.9633073210716248),
 ('scary.', 0.9455533027648926),
 ('bad', 0.9406482577323914),
 ('good', 0.9352386593818665),
 ('too', 0.9335749745368958),
 ('pretty', 0.9307319521903992),
 ('not', 0.9262821674346924),
 ("it's", 0.9235444664955139),
 ('nothing', 0.9228380918502808),
 ('done', 0.9198395013809204)]

In [180]:
# Making Lemmatizing object
lem = WordNetLemmatizer()
# Using Porter Stemmer
p_stem = PorterStemmer()

# Traversing whole dataset
for i in tqdm(range(len(df['review']))):
    # Tokenization
    tokens = word_tokenize(str(df['review'][i]))
    # Converting all characters to lower case
    tokens = [w.lower() for w in tokens]
    # Remove all punctuations from sentenses
    tokens = [re_puncs.sub('', w) for w in tokens]
    # Checking all words is alphabets or not
    tokens = [i for i in tokens if i.isalpha()]
    # Removing all stop words from the sentenses
    tokens = [w for w in tokens if w not in stop_word]
    # Doing Lemmatizing of words
    tokens = [lem.lemmatize(w) for w in tokens]
    # Stemming process
    tokens = [p_stem.stem(w) for w in tokens]
    # Finally convert to string
    r = ' '.join(tokens)
    # Storing the final string into main list
    main.append(r)


100%|██████████| 1485/1485 [00:22<00:00, 65.99it/s] 


In [181]:
df['translated'] = main

In [182]:
df

,title,review,rating,translated
0,Don't dare blink as you may miss something.,"Wong Kar Mun went blind at the age of two, 18 ...",9/10,wong kar mun went blind age two year later und...
1,Excellent horror film with a weak ending,Mun (Angelica Lee) is a young woman who has be...,7/10,mun angelica lee young woman blind sinc two ye...
2,A Corneal Transplant from the Twilight Zone.,This is not the first time that a movie where ...,8/10,first time movi main charact get corneal trans...
3,"One of the year's best horror movies, though t...",Even the website of this movie gave me the cre...,8/10,even websit movi gave creep turn one scariest ...
4,Good and scary!!!,"Of all the horror movie genres in existence, g...",10/10,horror movi genr exist ghost stori alway perso...
...,...,...,...,...
1480,A rewarding experience,A Tale Of Two Sisters is a heart-wrenching sto...,10/10,tale two sister heartwrench stori psycholog ho...
1481,"Creepy, Cringe, Disturbing,Twisted",Endless expectation for something to happen. C...,6/10,endless expect someth happen cring random stuf...
1482,It will make you scream like a teenage girl.,I saw this movie while I was in London a few m...,9/10,saw movi london month back nt know anyth movi ...
1483,Not as good as I hoped,I watched this film as part of the EXTREME ASI...,10/10,watch film part extrem asia season local cinem...


In [183]:
#df['translated'][88]

In [184]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['translated'])

In [185]:
print(len(tokenizer.word_counts))

12200


In [186]:
counter = 0
for key, value in tokenizer.word_counts.items():
    if value > 2400:
        print(key, value)
        counter+=1

print(counter)

film 3687
nt 2443
movi 3731
3


**grupa 1** fright(ened), shocked/shocking, horror/horrified, horrendous, overwhelming, terror/terrified, disturbing, bloody, disgust(ing), brutal, gory/gore, barbarous, repulsed/-ing, gross(ed out)

In [187]:
count1 = 0
num=tokenizer.word_counts['fright']
print(num)
count1+=num

37


In [188]:
num=tokenizer.word_counts['frighten']
print(num)
count1+=num

210


In [189]:
num=tokenizer.word_counts['shock']
print(num)
count1+=num

175


In [190]:
num=tokenizer.word_counts['horrifi']
print(num)
count1+=num

46


In [191]:
num=tokenizer.word_counts['horrif']
print(num)
count1+=num

37


In [192]:
num=tokenizer.word_counts['horrend']
print(num)
count1+=num

3


In [193]:
num=tokenizer.word_counts['overwhelm']
print(num)
count1+=num

15


In [194]:
num=tokenizer.word_counts['overwhelming']
print(num)
count1+=num

KeyError: 'overwhelming'

In [195]:
num=tokenizer.word_counts['disturb']
print(num)
count1+=num

127


In [196]:
num=tokenizer.word_counts['terror']
print(num)
count1+=num

114


In [197]:
num=tokenizer.word_counts['blood']
print(num)
count1+=num

88


In [198]:
num=tokenizer.word_counts['terrifi']
print(num)
count1+=num

191


In [199]:
num=tokenizer.word_counts['disgust']
print(num)
count1+=num

9


In [200]:
num=tokenizer.word_counts['brutal']
print(num)
count1+=num

11


In [201]:
num=tokenizer.word_counts['gore']
print(num)
count1+=num

139


In [202]:
num=tokenizer.word_counts['repuls']
print(num)
count1+=num

3


In [203]:
num=tokenizer.word_counts['gross']
print(num)
count1+=num

4


In [204]:
print('WYNIK DLA GRUPY PIERWSZEJ TOOOOO',count1)

WYNIK DLA GRUPY PIERWSZEJ TOOOOO 1209


**grupa 2** afraid, alarmed, threatening, hair-rising, distress, helpless(ness), anxiety/anxious, worried, apprehension, uneasy, chilling, eerie, intimidating, unnerving, appalling

In [205]:
count2=0
num=tokenizer.word_counts['distress']
print(num)
count2+=num

5


In [206]:
num=tokenizer.word_counts['afraid']
print(num)
count2+=num

44


In [207]:
num=tokenizer.word_counts['alarm']
print(num)
count2+=num

3


In [208]:
num=tokenizer.word_counts['threaten']
print(num)
count2+=num

21


In [209]:
num=tokenizer.word_counts['anxious']
print(num)
count2+=num

KeyError: 'anxious'

In [210]:
num=tokenizer.word_counts['anxieti']
print(num)
count2+=num

8


In [211]:
num=tokenizer.word_counts['worri']
print(num)
count2+=num

18


In [212]:
num=tokenizer.word_counts['apprehens']
print(num)
count2+=num

3


In [213]:
num=tokenizer.word_counts['uneasi']
print(num)
count2+=num

30


In [214]:
num=tokenizer.word_counts['chilli']
print(num)
count2+=num

1


In [215]:
num=tokenizer.word_counts['eeri']
print(num)
count2+=num

117


In [216]:
num=tokenizer.word_counts['helpless']
print(num)
count2+=num

10


In [217]:
num=tokenizer.word_counts['unnerv']
print(num)
count2+=num

37


In [218]:
num=tokenizer.word_counts['appal']
print(num)
count2+=num

2


In [219]:
num=tokenizer.word_counts['fear']
print(num)
count2+=num

188


In [220]:
print('WYNIK DLA GRUPY DRUGIEJ TOOOOO',count2)

WYNIK DLA GRUPY DRUGIEJ TOOOOO 487
